# LIBRARIES

In [1]:
import os
import ast
import math
import ccxt
import time
import pickle
import numpy as np
import configparser
import pandas as pd
import google.cloud
from pandas_gbq import gbq
import matplotlib.cm as cm
from boruta import BorutaPy
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from google.cloud import bigquery
from sklearn import preprocessing
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from scipy.interpolate import UnivariateSpline
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#_____

pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None

# FUNCTIONS

In [2]:
#FUNCIÓN QUE LEE EL ARCHIVO CONFIG
def get_config(category, key):
    
    global config
    return config[category][key]

#_____

#FUNCIÓN QUE GENERA CONEXIÓN CON ARCHIVO CONFIG
def updateConfig(config_name):
    
    global config
    
    config = configparser.ConfigParser()
    config.sections()
    config.read(config_name)
    
#_____

#FUNCIÓN QUE GRAFICA POR COLORES SEGUN COMPRAS Y NO COMRPAS
def plot_colourline(x,y,c):
    c = c
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i])
    return

#_____

#FUNCIÓN QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):
    while True:
        try:
            return pickle.load(open(variable_name+".pickle","rb"))
            break
        except:
            time.sleep(1)

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ABAJO SEGÚN DECIMALES
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factorho,

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ARRIBA SEGÚN DECIMALES
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

#_____

def cargarPortafolio():

    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')

    #_____CREAR PORTAFOLIO
    portfolio=readPickleVariable("portfolio")
    portfolio.reset_index(inplace=True,drop=True)
    portfolio.RSI=portfolio.RSI*(-1)
    
    if int(get_config("ITERATION","TREND"))==1:
        try:
            portfolio=portfolio.loc[(portfolio.TREND>0)]
            portfolio.reset_index(inplace=True,drop=True)
        except:
            pass
    if int(get_config("ITERATION","AMPLITUD"))==1:
        try:
            portfolio=portfolio.loc[(portfolio.AMPLITUD>0)]
            portfolio.reset_index(inplace=True,drop=True)
        except:
            pass
    if int(get_config("ITERATION","RENTABILIDAD"))==1:
        try:
            portfolio=portfolio.loc[(portfolio.RENTABILIDAD>0)]
            portfolio.reset_index(inplace=True,drop=True)
        except:
            pass
    if int(get_config("ITERATION","LIMIT"))==1:
        try:
            portfolio=portfolio.loc[(portfolio.LIMIT>0)]
            portfolio.reset_index(inplace=True,drop=True)
        except:
            pass
    if int(get_config("ITERATION","VOLUME"))==1:
        try:
            portfolio=portfolio.loc[(portfolio.VOLUME>0)]
            portfolio.reset_index(inplace=True,drop=True)
        except:
            pass
    try:
        portfolio=portfolio.loc[(portfolio.RSI<=int(get_config("ITERATION","RSI")))]
        portfolio.reset_index(inplace=True,drop=True)
    except:
        pass

    #_____PORTAFOLIO
    return portfolio

#_____

#FUNCIÓN QUE RETORNA BALANCE USDC EN BINANCE
def getUsdtBinance():
    global binance
    while True:
        try:
            return float(binance.fetch_balance()["USDT"]["total"])
            break
        except:
            print("ERROR: getUsdtBinance()")
            time.sleep(1)

#_____

#FUNCIÓN QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(market_model,time,rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

    #_____SI NO SE DA UN NÚMERO ESPECÍFICO DE FILAS POR PARÁMETRO
    if rows==None:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC",project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    else:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    candlesDataBase_BigQuery.OPEN=candlesDataBase_BigQuery.OPEN.astype(float)
    candlesDataBase_BigQuery.HIGH=candlesDataBase_BigQuery.HIGH.astype(float)
    candlesDataBase_BigQuery.LOW=candlesDataBase_BigQuery.LOW.astype(float)
    candlesDataBase_BigQuery.CLOSE=candlesDataBase_BigQuery.CLOSE.astype(float)
    candlesDataBase_BigQuery.VOLUME=candlesDataBase_BigQuery.VOLUME.astype(float)

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    if time != "1":

        #_____UPDATE CONFIG PREDICTION
        updateConfig('/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini')

        #_____CREAR COLUMNAS
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery.copy()
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery_copy.set_index('TIME')
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy[["ID"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min()
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["VOLUME"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).sum())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["LOW"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["HIGH"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).max())

        #_____AGREAR DATOS
        for i in list(candlesDataBase_BigQuery_copy_total.index.values):
            try:
                candlesDataBase_BigQuery_copy_total.at[i,"OPEN"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==i]["OPEN"].values[0]
                final_time = i + np.timedelta64(int(time),'m')
                if final_time in list(candlesDataBase_BigQuery_copy_total.index.values):
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==final_time]["OPEN"].values[0]
                else:
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"CLOSE"]
            except:
                pass

        #_____RESET INDEX
        candlesDataBase_BigQuery_copy_total.reset_index(level=['TIME'],inplace=True)

        #_____COLUMNA MARKET
        for i in range(0,len(candlesDataBase_BigQuery_copy_total)):
            candlesDataBase_BigQuery_copy_total.at[i,"MARKET"]=candlesDataBase_BigQuery.MARKET.values[0]

        #_____ORDENAR COLUMNAS
        column_names = list(candlesDataBase_BigQuery.columns)
        candlesDataBase_BigQuery_copy_total = candlesDataBase_BigQuery_copy_total.reindex(columns=column_names)
        candlesDataBase_BigQuery_copy_total.dropna(inplace=True)
        candlesDataBase_BigQuery=candlesDataBase_BigQuery_copy_total.copy()

    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI(candlesDataBase_BigQuery):
    
    global config
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    series=pd.Series(list(candlesDataBase_BigQuery.CLOSE.values))
    period=int(get_config("PARAMETERS","RSI_LEN"))
    
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    
    rsi=100 - 100 / (1 + rs)
    
    for i in list(rsi.index.values):
        candlesDataBase_BigQuery.at[i,"RSI"]=rsi[i]
                                        
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI_GENERAL(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

#_____

#FUNCIÓN QUE CALCULA MACD
def MACD(candlesDataBase_BigQuery):
    
    global config
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]
    
    #_____CREAR VARIABLE MACD
    macd_12 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_SHORT")), adjust=False).mean()
    macd_26 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_LONG")), adjust=False).mean()
    macd = macd_12 - macd_26
    macd_9 = macd.ewm(span=int(get_config("PARAMETERS","MACD_TRIGGER")), adjust=False).mean()

    #_____AGREGAR VARIABLES A LA BASE DE DATOS
    candlesDataBase_BigQuery["MACD"]=macd
    #candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")]=macd_12
    #candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")]=macd_26
    candlesDataBase_BigQuery["MACD_TRIGGER"]=macd_9
    
    return candlesDataBase_BigQuery

#_____



#_____

#FUNCIÓN QUE BRINDA LOS NIVELES DE FIBONACCI
def fibonacciLevels(candlesDataBase_BigQuery):
    
    global config
    global fractal
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    #_____ACTUALIZAR PARÁMETROS
    FIBONACCI_PERIOD=int(int(get_config("PARAMETERS","FIBONACCI_PERIOD"))/int(fractal))

    #_____RECORRER DATOS DE TEST
    for i in range(FIBONACCI_PERIOD,len(candlesDataBase_BigQuery)):
        
        #_____FIBONACCI
        lows=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].LOW.values)
        highs=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].HIGH.values)
        Price_Min = min(lows)
        Price_Max = max(highs)
        Diff = Price_Max-Price_Min
        level1 = Price_Max - 0.236 * Diff
        level2 = Price_Max - 0.382 * Diff
        level3 = Price_Max - 0.618 * Diff
        level4 = Price_Max - 0.786 * Diff
        
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_1"]=level1
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_2"]=level2
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_3"]=level3
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_4"]=level4
            
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CREA VARIABLES ENCONTRADAS EN ARTÍCULOS
def newVariables(candlesDataBase_BigQuery):
    
    #_____VARIABLES NUEVAS
    candlesDataBase_BigQuery['SMA5_VOL'] = candlesDataBase_BigQuery.groupby('MARKET')['VOLUME'].transform(lambda x: x.rolling(window = 5).mean())
    candlesDataBase_BigQuery['SMA15_VOL'] = candlesDataBase_BigQuery.groupby('MARKET')['VOLUME'].transform(lambda x: x.rolling(window = 15).mean())
    candlesDataBase_BigQuery['SMA_VOL_RATIO'] = candlesDataBase_BigQuery['SMA5_VOL']/candlesDataBase_BigQuery['SMA15_VOL']

    #_____VARIABLES NUEVAS
    candlesDataBase_BigQuery['LOWEST_5D'] = candlesDataBase_BigQuery.groupby('MARKET')['LOW'].transform(lambda x: x.rolling(window = 5).min())
    candlesDataBase_BigQuery['HIGH_5D'] = candlesDataBase_BigQuery.groupby('MARKET')['HIGH'].transform(lambda x: x.rolling(window = 5).max())
    candlesDataBase_BigQuery['LOWEST_15D'] = candlesDataBase_BigQuery.groupby('MARKET')['LOW'].transform(lambda x: x.rolling(window = 15).min())
    candlesDataBase_BigQuery['HIGH_15D'] = candlesDataBase_BigQuery.groupby('MARKET')['HIGH'].transform(lambda x: x.rolling(window = 15).max())
    candlesDataBase_BigQuery['STOCHASTIC_5'] = ((candlesDataBase_BigQuery['CLOSE'] - candlesDataBase_BigQuery['LOWEST_5D'])/(candlesDataBase_BigQuery['HIGH_5D'] - candlesDataBase_BigQuery['LOWEST_5D']))*100
    candlesDataBase_BigQuery['STOCHASTIC_15'] = ((candlesDataBase_BigQuery['CLOSE'] - candlesDataBase_BigQuery['LOWEST_15D'])/(candlesDataBase_BigQuery['HIGH_15D'] - candlesDataBase_BigQuery['LOWEST_15D']))*100
    candlesDataBase_BigQuery['STOCHASTIC_D_5'] = candlesDataBase_BigQuery['STOCHASTIC_5'].rolling(window = 5).mean()
    candlesDataBase_BigQuery['STOCHASTIC_D_15'] = candlesDataBase_BigQuery['STOCHASTIC_5'].rolling(window = 15).mean()
    candlesDataBase_BigQuery['STOCHASTIC_RATIO'] = candlesDataBase_BigQuery['STOCHASTIC_D_5']/candlesDataBase_BigQuery['STOCHASTIC_D_15']
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN SUPERTREND
def superTrend(df, atr_period, multiplier):
    
    high = df['HIGH']
    low = df['LOW']
    close = df['CLOSE']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    #return pd.DataFrame({
    #    'Supertrend': supertrend,
    #    'Final Lowerband': final_lowerband,
    #    'Final Upperband': final_upperband
    #}, index=df.index)

    df["SUPERTREND"]=supertrend

    return df

#_____

def suaviPlusDerivates(candlesDataBase_BigQuery):
    
    global config

    #_____UPDATE CONFIG PREDICTION
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")

    short_ma_hyper=int(get_config("PARAMETERS","SHORT_MA_HYPER"))
    short_ma_normal=int(get_config("PARAMETERS","SHORT_MA_NORMAL"))
    long_ma_normal=int(get_config("PARAMETERS","LONG_MA_NORMAL"))
    long_ma_hyper=int(get_config("PARAMETERS","LONG_MA_HYPER"))
    ultra_long_ma_normal=int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))

    #_____VECTOR DE PRECIOS
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]

    #_____SUAVIZACIONES MA & EMA
    close_rolling_short_ma_hyper=close_data_df.rolling(short_ma_hyper).mean()
    close_rolling_short_ma_normal=close_data_df.rolling(short_ma_normal).mean()
    close_rolling_long_ma_normal=close_data_df.ewm(span=long_ma_normal).mean()
    close_rolling_long_ma_hyper=close_data_df.ewm(span=long_ma_hyper).mean()
    close_rolling_ultra_long_ma_normal=close_data_df.ewm(span=ultra_long_ma_normal).mean()

    #_____INSERTAR NUEVAS VARIABLES
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_HYPER"]=close_rolling_short_ma_hyper
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_NORMAL"]=close_rolling_short_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_NORMAL"]=close_rolling_long_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_HYPER"]=close_rolling_long_ma_hyper
    candlesDataBase_BigQuery["ROLLING_ULTRA_LONG_MA_NORMAL"]=close_rolling_ultra_long_ma_normal

    #_____BOLLINGER
    candlesDataBase_BigQuery['15MA'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.rolling(window=15).mean())
    candlesDataBase_BigQuery['SD'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.rolling(window=15).std())
    candlesDataBase_BigQuery['UPPERBAND'] = candlesDataBase_BigQuery['15MA'] + 2*candlesDataBase_BigQuery['SD']
    candlesDataBase_BigQuery['LOWERBAND'] = candlesDataBase_BigQuery['15MA'] - 2*candlesDataBase_BigQuery['SD']

    #_____RATE CHANGE
    candlesDataBase_BigQuery['RC'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.pct_change(periods = 15))

    #_____ELIMINAR DATOS FALTANTES
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    #####

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_CLOSE= np.arange(len(candlesDataBase_BigQuery))
    y_spl_CLOSE = UnivariateSpline(x_CLOSE,candlesDataBase_BigQuery[["CLOSE"]],s=0,k=3)
    Y_1D_CLOSE=y_spl_CLOSE.derivative(n=1)
    Y_1D_CLOSE=pd.DataFrame(Y_1D_CLOSE(x_CLOSE))
    candlesDataBase_BigQuery["Y_1D_CLOSE"]=Y_1D_CLOSE

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_HYPER = UnivariateSpline(x_ROLLING_SHORT_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_HYPER=y_spl_ROLLING_SHORT_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_HYPER(x_ROLLING_SHORT_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_HYPER"]=Y_1D_ROLLING_SHORT_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_NORMAL = UnivariateSpline(x_ROLLING_SHORT_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_NORMAL=y_spl_ROLLING_SHORT_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_NORMAL(x_ROLLING_SHORT_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_NORMAL"]=Y_1D_ROLLING_SHORT_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_NORMAL=y_spl_ROLLING_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_LONG_MA_NORMAL(x_ROLLING_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_NORMAL"]=Y_1D_ROLLING_LONG_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_HYPER = UnivariateSpline(x_ROLLING_LONG_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_LONG_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_HYPER=y_spl_ROLLING_LONG_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_LONG_MA_HYPER(x_ROLLING_LONG_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_HYPER"]=Y_1D_ROLLING_LONG_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_ULTRA_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_ULTRA_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_ULTRA_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_ULTRA_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=y_spl_ROLLING_ULTRA_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL(x_ROLLING_ULTRA_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL"]=Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL

    #####

    #_____CALCULAR VARIABLE RSI
    RSI_ROLLING_SHORT_MA_HYPER = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_SHORT_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_HYPER = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_ULTRA_LONG_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_ULTRA_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))

    #_____AGRAGAR VARIABLE RSI A BASE DE DATOS
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_HYPER"]=RSI_ROLLING_SHORT_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_NORMAL"]=RSI_ROLLING_SHORT_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_NORMAL"]=RSI_ROLLING_LONG_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_HYPER"]=RSI_ROLLING_LONG_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_ULTRA_LONG_MA_NORMAL"]=RSI_ROLLING_ULTRA_LONG_MA_NORMAL

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))+1:]
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CALCULA MACD
def LAGS(candlesDataBase_BigQuery):
    
    for j in list(candlesDataBase_BigQuery.columns)[3:]:
        candlesDataBase_BigQuery[j+"_1"]=candlesDataBase_BigQuery[j].shift(1)
        candlesDataBase_BigQuery[j+"_2"]=candlesDataBase_BigQuery[j].shift(2)
        candlesDataBase_BigQuery[j+"_3"]=candlesDataBase_BigQuery[j].shift(3)
        
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    return candlesDataBase_BigQuery

#_____

#CREATE VARIABLES
def variableCreation(candlesDataBase_BigQuery):
    
    global fractal
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    candlesDataBase_BigQuery=RSI(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=MACD(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=fibonacciLevels(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=newVariables(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=superTrend(candlesDataBase_BigQuery, int(get_config("PARAMETERS","ATR_PERIOD")), int(get_config("PARAMETERS","ATR_MULTIP")))
    candlesDataBase_BigQuery=suaviPlusDerivates(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=LAGS(candlesDataBase_BigQuery)
    
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE ESCALA LAS VARIABLES DEL TEST CON LOS PARÁMETROS DEL TRAIN (MEDIA + SDT)
def scaleTestVariables(candlesDataBase_BigQuery):
    
    global marketDict
    
    marketDict=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/PREDICTION/marketDict")
    
    #_____ESCALAR VARIABLES
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:]
    marketVariables=list(candlesDataBase_BigQuery.columns)[0:3]

    #_____LISTA APPEND DE VARIABLES
    listaAppend=[]

    for i in continuous_variables:

        try:

            STD=marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["STD"]
            MEAN=marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["MEAN"]
            
            candlesDataBase_BigQuery[str(i)] = candlesDataBase_BigQuery[str(i)]-MEAN
            candlesDataBase_BigQuery[str(i)] = candlesDataBase_BigQuery[str(i)]/STD
            listaAppend.append(i)

        except:

            pass

    candlesDataBase_BigQuery=candlesDataBase_BigQuery[marketVariables+listaAppend]

    #_____RETURN
    return candlesDataBase_BigQuery
    
#_____

#FUNCIÓN QUE PREDICE SOBRE LOS DATOS TEST
def predictFunction(myData_DataFrame):

    global market_Model
    global market_Parameters
    
    #_____COLUMNAS VARIABLE X
    continuous_variables=ast.literal_eval(market_Parameters.at[0,"TRAIN_COLUMNS"])
    
    #_____DATOS
    X_test=myData_DataFrame.copy()
    X_test=X_test[continuous_variables]
    X_test=X_test.values
    
    #_____PREDICTION
    prediction = market_Model.predict(np.array(X_test))
    
    #_____RETURN
    return prediction

#_____

#FUNCIÓN QUE CORRIGE PREDICCIÓN DADO QUE EL PRECIO EN EL QUE ME ENCUENTRO ES MENOR AL PRECIO ANTERIOR
def correctPredict(test_sample,PERIODS_EVALUATION_PAST,MAX_PERIODS_EVALUATION_QUANTILE,PERIODS_LINEAREG_BEFORE):
    
    global myRSI
    
    #_____ACTUALIZAR CONFIG
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    #_____VARAIABLE QUE PARAMETRIZA PSADO DE EVALUACIÓN
    past_evaluation=PERIODS_EVALUATION_PAST
    
    #_____QUANLIT DE AVIALUACIÓN DE PRECIO MÁXIMO
    max_past_evaluation_quantile=MAX_PERIODS_EVALUATION_QUANTILE
    
    #_____SI RSI ES MAYOR A THRESHOLD, NO COMPRAR
    for i in range(0,len(test_sample)):
        if (test_sample.at[i,"PRED"]=="SI") and (myRSI[i]>float(get_config("PARAMETERS","MY_RSI"))):
            test_sample.at[i,"PRED"]="NO"
    
    #_____RECORRER DATOS DE TEST
    for i in range(past_evaluation,len(test_sample)):
        
        #_____SI LA PREDICCIÓN FUE COMPRAR & SI CLOSE ANTERIOR ES MAYOR
        if (test_sample.at[i,"PRED"]=="SI") and (test_sample.at[i,"CLOSE"]>=np.quantile(test_sample[i-past_evaluation:i].CLOSE.values,max_past_evaluation_quantile)):
            
            #_____CONVERTIR OBSERVACIÓN EN NO COMRPAR
            test_sample.at[i,"PRED"]="NO"
            
    #_____RECORRER DATOS DE TEST
    periods_lineareg=PERIODS_LINEAREG_BEFORE
    for i in range(periods_lineareg,len(test_sample)):
            
        if (test_sample.at[i,"PRED"]=="SI"):

            periods_lineareg=PERIODS_LINEAREG_BEFORE

            X=np.arange(periods_lineareg).reshape(-1, 1)
            Y=test_sample[i-periods_lineareg:i].CLOSE.values.reshape(-1, 1)

            linear_regressor = LinearRegression()
            nuevaVariable=linear_regressor.fit(X, Y)
            Y_pred = linear_regressor.predict(X)

            #_____CORREGIR
            if nuevaVariable.coef_ < 0:

                #_____CONVERTIR OBSERVACIÓN EN NO COMRPAR
                test_sample.at[i,"PRED"]="NO"
                
    #_____REESTRUCTURAR TEST SAMPLE
    test_sample=test_sample[max(past_evaluation,periods_lineareg)+1:]
    test_sample.reset_index(inplace=True,drop=True)
                
    return test_sample

#_____

#FUNCIÓN QUE DECIDE SI COMPRO O NO COMPRO
def shouldIMyFriend(fractal):
    
    global myRSI
    global market
    global market_Model
    global market_Parameters

    #_____BIANNCE CANDLES LIST
    binanceCandleList=[1,3,5,15,30]

    #_____DESCARGAR DATOS DEL MOMENTO
    myData = binance.fetch_ohlcv(market,str(min(binanceCandleList, key=lambda x:abs(x-int(fractal))))+"m",limit=1000)

    #_____CREAR DATAFRAME
    myData_DataFrame =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

    #_____LLENAR BASE DE DATOS
    for i in range(0,len(myData)):
        myData_DataFrame.at[i,"ID"]=myData[i][0]
        myData_DataFrame.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
        myData_DataFrame.at[i,"MARKET"]=market
        myData_DataFrame.at[i,"OPEN"]=myData[i][1]
        myData_DataFrame.at[i,"HIGH"]=myData[i][2]
        myData_DataFrame.at[i,"LOW"]=myData[i][3]
        myData_DataFrame.at[i,"CLOSE"]=myData[i][4]
        myData_DataFrame.at[i,"VOLUME"]=myData[i][5]

    #_____CALCULAR TODAS LAS VARIABLES
    myData_DataFrame=variableCreation(myData_DataFrame)
    
    #_____RSI
    myRSI=list(myData_DataFrame.RSI.values)

    #_____ESCALAR VARIABLES
    myData_DataFrame=scaleTestVariables(myData_DataFrame)

    #_____DESCARGAR MODELO DE PREDICCIÓN DEL MERCADO
    market_Model=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/PREDICTION/train_model")

    #_____DESCARGAR PARÁMETROS DE CORRECCIÓN
    market_Parameters=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/"+market.replace("/","_")+"_PARAMETERS"+"_"+fractal)
    
    #_____PREDECIR
    myData_DataFrame["PRED"]=predictFunction(myData_DataFrame)

    #_____ASIGNAR VALORES DE CORRECCIÓN
    PERIODS_EVALUATION_PAST=int(market_Parameters.at[0,"PERIODS_EVALUATION_PAST"])
    MAX_PERIODS_EVALUATION_QUANTILE=market_Parameters.at[0,"MAX_PERIODS_EVALUATION_QUANTILE"]
    PERIODS_LINEAREG_BEFORE=int(market_Parameters.at[0,"PERIODS_LINEAREG_BEFORE"])

    #_____CORREGIR PREDICCIÓN
    myData_DataFrame=correctPredict(myData_DataFrame,PERIODS_EVALUATION_PAST,MAX_PERIODS_EVALUATION_QUANTILE,PERIODS_LINEAREG_BEFORE)

    #_____RETURN
    if market_Parameters.at[0,"NET_UTILITY"]==0:
        return "NO"
    else:
        return myData_DataFrame.at[len(myData_DataFrame)-1,"PRED"]
    
#_____

#FUNCIÓN QUE ESTIMA LA CANTIDAD DE DECIMALES A REDONDEAR
def marketFloats(value): 
    Completo = str(value)
    Entero= str(math.floor(value))
    lenCompleto = len(Completo)
    lenEntero = len(Entero)
    if lenEntero != lenCompleto:
        decimalVector=Completo.split(".")
        for i in decimalVector[1]:
            if i != "0":
                tengoDecimales=True
            else:
                tengoDecimales=False
        if tengoDecimales:
            decimales=lenCompleto-lenEntero-1
        else:
            decimales=0
    else:
        decimales = 0
    
    return decimales

#_____

#FUNCIÓN QUE SIMULA COMPRA/VENTA MARKET EN BINANCE SEGÚN MERCADO -> SI SIDE==BUY, TRADINGAMOUNT==USDT // SI SIDE==SELL, TRADINGAMOUNT==CRYPTO
def simulateMarket(exchange,market,tradingAmount,side):
    
    global comission
    global decimales
    
    #_____ACTUALIZAR CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____CHECKEAR PARA PRIMERA ITERACIÓN Y EVITAR NAN
    if math.isnan(comission):
        comission=float(get_config("BINANCE","BUY_MARKET_FEE"))
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    while True:
        
        try:
    
            #EN CASO DE QUE SEA COMPRA
            if side=="buy":

                #VALIDAR QUE ORDERBOOK ALCANCE PARA HACER LA SIMULACIÓN
                while_break=0
                while while_break<tradingAmount*1.5:
                    #DESCARGAR BUYINGBOOK
                    buyingBook=pd.DataFrame(exchange.fetch_order_book(market)["asks"],columns=["ASK_VALUE","ASK_SIZE"])
                    buyingBook["ASK_PRICE"]=buyingBook["ASK_VALUE"]*buyingBook["ASK_SIZE"]

                    #ACTUALIZAR SUMA DE VOLUMENES DE ORDENES EN ORDERBOOK DESCARGADO
                    while_break=sum(buyingBook.ASK_PRICE.values)

                    #SI VOLUMEN TOTAL ES MAYOR A LO BUSCADO, SEGUIR CON SIMULACIÓN -> DLC, PRINT + ESPERAR
                    if while_break<tradingAmount*1.5:
                        print("[ERROR]: buyingBook(binance,"+market+","+str(tradingAmount)+")")
                        time.sleep(1)

                #_____

                #SIMULACIÓN DE COMPRA CON USDT -> RETORNA VALOR DE CRYPTO ADQUIRIDO NETO
                usdt_used=0
                crypto_buyed=0
                usdt_used_acum=0
                crypto_buyed_acum=0
                contador_buying_book=0

                while usdt_used_acum<tradingAmount:

                    usdt_used=buyingBook.at[contador_buying_book,"ASK_PRICE"]
                    crypto_buyed=buyingBook.at[contador_buying_book,"ASK_SIZE"]
                        
                    #_____SIMULACIÓN
                    if usdt_used_acum+usdt_used < tradingAmount:
                        usdt_used_acum=usdt_used_acum+usdt_used
                        crypto_buyed_acum=crypto_buyed_acum+crypto_buyed
                    else:
                        diferencia=tradingAmount-usdt_used_acum
                        usdt_used_acum=usdt_used_acum+diferencia

                        porcentaje_diferencia=diferencia/usdt_used
                        crypto_buyed_acum=crypto_buyed_acum+crypto_buyed*porcentaje_diferencia
                        
                
                #_____REVISAR DECIMALES
                decimales=0
                for i in range(0,10):
                    crypto_buyed=buyingBook.at[i,"ASK_SIZE"]
                    if marketFloats(crypto_buyed)>decimales:
                        decimales=marketFloats(crypto_buyed)

                #crypto_buyed_acum=round_decimals_down(crypto_buyed_acum*(1-comission),decimales)
                crypto_buyed_acum=round_decimals_down(crypto_buyed_acum,decimales)
                
                response = crypto_buyed_acum

            #_____

            #EN CASO DE QUE SEA VENTA
            if side=="sell":

                #VALIDAR QUE ORDERBOOK ALCANCE PARA HACER LA SIMULACIÓN
                while_break=0
                while while_break<tradingAmount*1.5:
                    #DESCARGAR SELLINGBOOK
                    sellingBook=pd.DataFrame(exchange.fetch_order_book(market)["bids"],columns=["BID_VALUE","BID_SIZE"])

                    #ACTUALIZAR SUMA DE VOLUMENES DE ORDENES EN ORDERBOOK DESCARGADO
                    while_break=sum(sellingBook.BID_SIZE.values)

                    #SI VOLUMEN TOTAL ES MAYOR A LO BUSCADO, SEGUIR CON SIMULACIÓN -> DLC, PRINT + ESPERAR
                    if while_break<tradingAmount*1.5:
                        print("[ERROR]: buyingBook("+exchange.name+","+market+","+str(tradingAmount)+")")
                        time.sleep(1)

                #_____

                #INICIALIZAR PARÁMETROS PARA LA SIMULACIÓN
                sellAmount=0.0
                sellCounter=0
                sellPriceACUM=0.0
                sellAmountACUM=0.0

                #SIMULAR VENTA
                while sellAmountACUM<tradingAmount:
                    sellAmount=float(sellingBook.at[sellCounter,"BID_SIZE"])
                    sellAmountACUM=sellAmountACUM+sellAmount
                    if sellAmountACUM<tradingAmount:
                        sellPriceACUM=sellPriceACUM+(sellAmount*float(sellingBook.iloc[sellCounter,0]))
                        sellCounter=sellCounter+1
                    else:
                        printCRY=sellAmountACUM-sellAmount+tradingAmount-(sellAmountACUM-sellAmount)
                        sellPriceACUM=sellPriceACUM+((tradingAmount-(sellAmountACUM-sellAmount))*float(sellingBook.iloc[sellCounter,0]))

                response = sellPriceACUM*(1-comission)
            
            break
            
        except Exception as e: 
            print(f"[ERROR]: simulateMarket() {e}")
            time.sleep(1)
            
    return response

#_____

#FUNCIÓN QUE CREA UNA COMPRA TIPO MARKET
def createMarketBuy(market,money):
    
    global amount
    global binance
    global marketBuyOrder
    
    amount=simulateMarket(binance,market,money,"buy")
    marketBuyOrder=binance.create_market_buy_order(symbol=market,amount=amount)
            
#_____

#FUNCIÓN QUE CREA ORDEN DE COMPRA MARKET Y ACTUALIZA BASE DE DATOS DE TRADEO
def buyMyFriend(market,money):
    
    global decimales
    global comission
    global row_index
    global market_TP
    global market_SL
    global cryptoAmount
    global marketBuyCost
    global stopSellPrice
    global marketBuyOrder
    global marketSellPrice
    global binance_machine
    global marketBuyOrder_ID
    global tradingBook_MonoBot_1
    
    #_____ACTUALIZAR CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____LEER BASE DE DATOS
    tradingBook_MonoBot_1=readPickleVariable("tradingBook_MonoBot_1")

    #_____DETERMINAR FILA DE INTEGRACIÓN DE DATOS
    row_index=len(tradingBook_MonoBot_1)

    #_____CREAR COMPRA MARKET
    createMarketBuy(market,money)

    #_____AGREGAR DATOS A BASE DE DATOS DE TRADEO
    tradingBook_MonoBot_1.at[row_index,"MACHINE"]=binance_machine
    tradingBook_MonoBot_1.at[row_index,"TRANSACTION_ID"]=marketBuyOrder["info"]["transactTime"]
    tradingBook_MonoBot_1.at[row_index,"MARKET"]=market
    tradingBook_MonoBot_1.at[row_index,"BUY_ID"]=marketBuyOrder["id"]
    tradingBook_MonoBot_1.at[row_index,"DATE_BUY"]=str(datetime.fromtimestamp(int(marketBuyOrder["info"]["transactTime"])/1000))
    tradingBook_MonoBot_1.at[row_index,"BUY_CRYPTO_AMOUNT"]=float(marketBuyOrder["info"]["executedQty"])
    
    try:
        tradingBook_MonoBot_1.at[row_index,"BUY_FEE"]=float(marketBuyOrder["fee"]["cost"])
        tradingBook_MonoBot_1.at[row_index,"BUY_CURRENCY_FEE"]=marketBuyOrder["fee"]["currency"]
    except:
        tradingBook_MonoBot_1.at[row_index,"BUY_FEE"]=float(get_config("BINANCE","BUY_MARKET_FEE"))*tradingBook_MonoBot_1.at[row_index,"BUY_CRYPTO_AMOUNT"]
        tradingBook_MonoBot_1.at[row_index,"BUY_CURRENCY_FEE"]=market
    
    
    #_____GUARDAR ID PARA FUTURO CRUCE CON VENTA
    marketBuyOrder_ID=marketBuyOrder["id"]
    
    #_____DESCARGAR PARÁMETROS DE ITERACIÓN
    market_Parameters=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/"+market.replace("/","_")+"_PARAMETERS"+"_"+get_config("ITERATION","BUY_CANDLE"))
    market_TP=float(market_Parameters.at[0,"TP"])
    market_SL=float(market_Parameters.at[0,"SL"])
    
    #_____ACTUALIZAR CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____ACTUALIZAR PRECIOS DE VENTA
    comission=float(get_config("BINANCE","BUY_MARKET_FEE"))
    tradingBook_MonoBot_1.at[row_index,"BUY_FEE_PERCET"]=comission
    tradingBook_MonoBot_1.at[row_index,"NET_BUY_COST"]=float(marketBuyOrder["cost"])*(1+comission)
    cryptoAmount=tradingBook_MonoBot_1.at[row_index,"BUY_CRYPTO_AMOUNT"]
    cryptoAmount=round_decimals_down(cryptoAmount,decimales)
    marketBuyCost=tradingBook_MonoBot_1.at[row_index,"NET_BUY_COST"]
    #marketSellPrice=tradingBook_MonoBot_1.at[row_index,"AVG_BUY_COST"]*(1+market_TP+comission)
    #stopSellPrice=tradingBook_MonoBot_1.at[row_index,"AVG_BUY_COST"]*(1-market_SL+comission)
    
#_____

#FUNCIÓN QUE RETORNA BALANCE USDC EN BINANCE
def getCryptoBinance(market):
    global binance
    
    my_market=market.replace("/USDT","")
    while True:
        try:
            return float(binance.fetch_balance()[my_market]["total"])
            break
        except:
            print("ERROR: getCryptoBinance()")
            time.sleep(1)
            
#_____

#FUNCIÓN QUE CREA ORDEN MARKET DE VENTA
def createMarketSell(market):
    
    global check
    global amount
    global binance
    global decimales
    global cryptoAmount
    global marketSellOrder
    
    check=getCryptoBinance(market)
    amount=round_decimals_down(getCryptoBinance(market),decimales)
    #marketSellOrder=binance.create_market_sell_order(symbol=market,amount=amount)
    #marketSellOrder=binance.create_market_sell_order(symbol=market,amount=cryptoAmount)
    try:
        marketSellOrder=binance.create_market_sell_order(symbol=market,amount=amount)
    except:
        if (getCryptoBinance(market)==0) and (type(marketSellOrder)==dict):
            pass
        else:
            ERROR_createMarketSell
    
#_____

#FUNCIÓN QUE CREA ORDEN DE VENTA MARKET Y ACTUALIZA BASE DE DATOS DE TRADEO
def sellMyFriend(market):
    
    global marketSellOrder
    global marketBuyOrder_ID
    global tradingBook_MonoBot_1

    #_____DETERMINAR FILA DE INTEGRACIÓN DE DATOS
    row_index=tradingBook_MonoBot_1.loc[tradingBook_MonoBot_1.BUY_ID==marketBuyOrder_ID].index.values[0]

    #_____CREAR VENTA MARKET
    createMarketSell(market)

    #_____AGREGAR DATOS A BASE DE DATOS DE TRADEO
    tradingBook_MonoBot_1.at[row_index,"SELL_ID"]=marketSellOrder["id"]
    tradingBook_MonoBot_1.at[row_index,"DATE_SELL"]=str(datetime.fromtimestamp(int(marketSellOrder["info"]["transactTime"])/1000))
    tradingBook_MonoBot_1.at[row_index,"SELL_FEE"]=float(marketSellOrder["fee"]["cost"])
    tradingBook_MonoBot_1.at[row_index,"SELL_CURRENCY_FEE"]=marketSellOrder["fee"]["currency"]
    tradingBook_MonoBot_1.at[row_index,"SELL_CRYPTO_AMOUNT"]=float(marketSellOrder["info"]["executedQty"])
    
    #_____ACTUALIZAR CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____CALCULAR COMISIÓN
    comission=float(get_config("BINANCE","BUY_MARKET_FEE"))
    tradingBook_MonoBot_1.at[row_index,"NET_SELL_COST"]=float(marketSellOrder["cost"])*(1-comission)
    tradingBook_MonoBot_1.at[row_index,"NET_UTILITY"]=tradingBook_MonoBot_1.at[row_index,"NET_SELL_COST"]/tradingBook_MonoBot_1.at[row_index,"NET_BUY_COST"]-1
    
    #_____ESCRIBIR PICKLE
    writePickleVariable(tradingBook_MonoBot_1,"tradingBook_MonoBot_1")
    
#_____

#FUNCIÓN QUE BALANCEA SALDO DE BNB PARA PAGO DE COMISIONES
def balanceBNB():

    if simulateMarket(binance,"BNB/USDT",getCryptoBinance("BNB")/int(get_config("ITERATION","MACHINES")),"sell") < myUSDT*0.01:
        #_____DECIMALES BNB
        buyingBook=pd.DataFrame(binance.fetch_order_book("BNB/USDT")["asks"],columns=["ASK_VALUE","ASK_SIZE"])
        decimales=0
        for i in range(0,10):
            crypto_buyed=buyingBook.at[i,"ASK_SIZE"]
            if marketFloats(crypto_buyed)>decimales:
                decimales=marketFloats(crypto_buyed)
        #_____VALIDAR MONTO
        amount_BNB=round_decimals_down(simulateMarket(binance,"BNB/USDT",max(myUSDT*0.01,11),"buy"),decimales)
        marketBuyOrder_BNB=binance.create_market_buy_order(symbol="BNB/USDT",amount=amount_BNB)
        
#_____

#FUNCIÓN QUE ACTUALIZA LOS MERCADOS DISPONIBLES
def uptadeAvailableMarkets():

    global activeMarkets_List
    
    #_____LEER MERCADOS ACTIVOS DE LOS DEMÁS BOTS
    while True:
        try:
            activeMarket_1=readPickleVariable("activeMarket_1")
            activeMarket_2=readPickleVariable("activeMarket_2")
            activeMarket_3=readPickleVariable("activeMarket_3")
            activeMarket_4=readPickleVariable("activeMarket_4")
            break
        except:
            time.sleep(1)
    
    #_____CREAR LISTA DE MERCADOS ACTIVOS
    activeMarkets_List=[activeMarket_1,activeMarket_2,activeMarket_3]

# PARAMETERS

In [3]:
#_____ACTUALIZAR ARCHIVO CONFIG
updateConfig('config_surfNet_MonoBot.ini')

#_____BINANCE
binance = ccxt.binance()
binance.apiKey = get_config('BINANCE', 'API_KEY_4')
binance.secret = get_config('BINANCE', 'API_SECRET_4')
binance.options['createMarketBuyOrderRequiresPrice'] = False

# START

In [4]:
#_____BASE DE DATOS LIMPIA
# tradingBook_MonoBot_4=pd.DataFrame(columns=["MACHINE","TRANSACTION_ID","MARKET","BUY_ID","DATE_BUY","SELL_ID","DATE_SELL","BUY_CRYPTO_AMOUNT","NET_BUY_COST","BUY_FEE","BUY_FEE_PERCET","BUY_CURRENCY_FEE","SELL_CRYPTO_AMOUNT","NET_SELL_COST","SELL_FEE","SELL_CURRENCY_FEE","NET_UTILITY"])
# writePickleVariable(tradingBook_MonoBot_4,"tradingBook_MonoBot_4")

# CHECKS

In [5]:
# #_____ID DE LA MAQUINA
# binance_machine=4

# #_____ACTUALIZAR CONFIG
# updateConfig('config_surfNet_MonoBot.ini')

# #_____INICIALIZAR COMISIÓN
# comission=float(get_config("BINANCE","BUY_MARKET_FEE"))

# #_____COMPRA-VENTA
# market="ZEC/USDT"
# money=100
# buyMyFriend(market,money)
# time.sleep(5)
# sellMyFriend(market)
# tradingBook_MonoBot_4

# RUN

In [6]:
#_____ID DE LA MAQUINA
binance_machine=4

#_____ACTUALIZAR CONFIG
updateConfig('config_surfNet_MonoBot.ini')

#_____INICIALIZAR COMISIÓN
comission=float(get_config("BINANCE","BUY_MARKET_FEE"))

#_____FRACTAL
fractal="15"

#_____TEXTO DE MERCADO ACTIVO
activeMarket_4=""
writePickleVariable(activeMarket_4,"activeMarket_4")

#_____WHILE INFINITO
while True:
    
    #_____TRY DE CONTINGENCIA
    #try:
    
    #_____CLEAR OUTPUT
    clear_output()
    
    #_____PRINT
    uptadeAvailableMarkets()
    print(activeMarket_4,activeMarkets_List)
    
    #_____ACTUALIZAR CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____SI QUIERO APAGARME
    if int(get_config("ITERATION","SHUTDOWN_4"))==1:
        
        #_____TEXTO DE MERCADO ACTIVO
        activeMarket_4=""
        writePickleVariable(activeMarket_4,"activeMarket_4")

        #_____BREAK
        break

    #_____CREAR PORTAFOLIO PARA ITERACIÓN
    portfolio=cargarPortafolio()
    
    #_____SI PORTAFOLIO TIENE POR LO MENOS UN MERCADO ABIERTO
    if len(portfolio)>=1:
        
        #_____EXIT PORTFOLIO
        exitPortfolio=0
        
        for market in list(portfolio.MARKET.values):
            
            if exitPortfolio==0:
            
                #_____ACTUALIZAR PORTAFOLIO DE EJECUCIÓN
                uptadeAvailableMarkets()

                #_____DESCARGAR PARÁMETROS DE CORRECCIÓN
                market_Parameters=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/"+market.replace("/","_")+"_PARAMETERS"+"_"+fractal)

                #_____RETURN
                if market_Parameters.at[0,"NET_UTILITY"]!=0:

                    #_____SI MERCADO ESTÁ DISPONIBLE
                    if market not in activeMarkets_List:

                        #_____TEXTO DE MERCADO ACTIVO
                        activeMarket_4=market
                        writePickleVariable(activeMarket_4,"activeMarket_4")

                        #_____CLEAR OUTPUT
                        clear_output()

                        #_____DEFINIR MERCADO
                        #market=list(portfolio.MARKET.values)[binance_machine-1]

                        #_____ACTUALIZAR CONFIG
                        updateConfig('config_surfNet_MonoBot.ini')

                        #_____REVISAR PRESUPUESTO
                        myUSDT=int(min(round_decimals_down(getUsdtBinance()-float(get_config("ITERATION","MIN_INVESTMENT")),0),float(get_config("ITERATION","MIN_INVESTMENT"))))

                        #_____SI CUENTO CON PRESUPUESTO MÍNIMO
                        if myUSDT>=float(get_config("ITERATION","MIN_INVESTMENT")):

                            #_____AJUSTAR PRESUPUESTO SI TENGO MAS DEL MÁXIMO
                            if myUSDT>=float(get_config("ITERATION","MAX_INVESTMENT")):

                                #_____NUEVO PRESUPUESTO
                                myUSDT=float(get_config("ITERATION","MAX_INVESTMENT"))

                            #_____SI MYUSDT ES MAYOR A MI PRESUPUESTO
                            if myUSDT>=float(get_config("ITERATION","BUDGET")):

                                #_____NUEVO PRESUPUESTO
                                myUSDT=float(get_config("ITERATION","BUDGET"))

                            #_____UPDATE CONFIG PREDICTION
                            updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")

                            #_____LISTA DE RESPUESTAS DE MODELOS
                            shouldIMy_List=[]

                            #_____LISTA DE FRACTALES
                            fractal_List=ast.literal_eval(get_config("PARAMETERS","FRACTAL_BUY"))

                            #_____ACTUALIZAR CONFIG
                            updateConfig('config_surfNet_MonoBot.ini')

                            #_____RECORRER FRACTALES
                            for fractal in fractal_List:

                                #_____DETERMINAR SI COMPRO O NO COMPRO
                                shouldIMy_List.append(shouldIMyFriend(fractal))

                            #_____PRINT
                            clear_output()
                            uptadeAvailableMarkets()
                            print(activeMarket_4,activeMarkets_List,shouldIMy_List)

                            #_____SI MERCADO ESTÁ DISPONIBLE
                            if market not in activeMarkets_List:

                                #_____SI DECIDO COMPRAR
                                if "NO" not in shouldIMy_List:

                                    #_____TEXTO DE MERCADO ACTIVO
                                    activeMarket_4=market
                                    writePickleVariable(activeMarket_4,"activeMarket_4")
                                    
                                    #_____PRINT
                                    clear_output()
                                    uptadeAvailableMarkets()
                                    print(activeMarket_4,activeMarkets_List,shouldIMy_List)

                                    #_____CREO COMPRA + VENTA (STOP LOSS) & ACTUALIZO BASE DE DATOS
                                    buyMyFriend(market,myUSDT)
                                    ejecuto_venta=0

                                    #_____NO PARAR HASTA VENDER
                                    while ejecuto_venta==0:
                                        
                                        #_____EXIT PORTFOLIO
                                        exitPortfolio=1

                                        #_____SLEEP
                                        time.sleep(int(get_config("ITERATION","SLEEP")))

                                        #_____ACTUALIZAR PRECIO SIMULADO DE VENTA                                                        
                                        precio_venta_simulado=simulateMarket(binance,market,cryptoAmount,"sell")
                                        margenUtilidad=(precio_venta_simulado/marketBuyCost)-1

                                        #_____PRINT
                                        clear_output()
                                        print(market,margenUtilidad)

                                        #_____SI MARGEN DE UTILIDAD CALCULADO LLEGA A ALGUNO DE LOS LÍMITES
                                        if (margenUtilidad >= market_TP) or (margenUtilidad <= -market_SL):

                                            #_____EJECUTAR ORDEN DE VENTA
                                            sellMyFriend(market)
                                            ejecuto_venta=1

                                            #_____CLEAR OUTPUT
                                            clear_output()

        #_____TEXTO DE MERCADO ACTIVO
        activeMarket_4=""
        writePickleVariable(activeMarket_4,"activeMarket_4")
        
        #_____PRINT
        clear_output()
        uptadeAvailableMarkets()
        print(activeMarket_4,activeMarkets_List)
        
    else:
        
        #_____TEXTO DE MERCADO ACTIVO
        activeMarket_4=""
        writePickleVariable(activeMarket_4,"activeMarket_4")
        
        #_____PRINT
        clear_output()
        uptadeAvailableMarkets()
        print(activeMarket_4,activeMarkets_List)
                        
    #_____SLEEP
    time.sleep(300)
    
    ##_____EXCEPT DE CONTINGENCIA
    #except Exception as e:
    #    
    #    #_____PRINT
    #    print(e)
    #    
    #    #_____BREAK
    #    break

/tmp/ipykernel_4901/1223752113.py:535: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  candlesDataBase_BigQuery[j+"_2"]=candlesDataBase_BigQuery[j].shift(2)
/tmp/ipykernel_4901/1223752113.py:536: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  candlesDataBase_BigQuery[j+"_3"]=candlesDataBase_BigQuery[j].shift(3)
/tmp/ipykernel_4901/1223752113.py:534: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

KeyError: "['MACD_12', 'MACD_26', 'MACD_12_1', 'MACD_12_2', 'MACD_12_3'] not in index"